<a href="https://colab.research.google.com/github/fatinirq/Spoiler-Review/blob/main/LSTM_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import spacy
import pandas as pd
from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset
from sklearn.model_selection import train_test_split
from bertModel import BERTGRU
import torch.nn as nn


In [ ]:
pathData="/content/drive/My Drive/SpoilerData/GoodReads/"
pathResutlts="/content/drive/My Drive/SpoilerData/GoodReads/BertResults"
data = [json.loads(line) for line in open('/content/drive/MyDrive/SpoilerData/CData.json', 'r')]
df=pd.DataFrame(data)

In [ ]:
df.head()

,review_sentences,user_id,timestamp,rating,has_spoiler,book_id,review_id
0,also high energy cosmic ray entering atmospher...,8842281e1d1347389f2ab93d60773d4d,2017-08-30,5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,avail free december http www audible com mt el...,8842281e1d1347389f2ab93d60773d4d,2017-03-22,3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,cheat system live choice learn daniela say lif...,8842281e1d1347389f2ab93d60773d4d,2017-03-20,3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,http www npr org recommended reading understan...,8842281e1d1347389f2ab93d60773d4d,2016-11-09,0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,surfer love end freya finding meaning surfing ...,8842281e1d1347389f2ab93d60773d4d,2016-04-25,4,True,25884323,332732725863131279a8e345b63ac33e


In [ ]:
listind=[]
count=0
for ind in df.index:
    if len(df['review_sentences'][ind].strip())==0:
        listind.append(ind)

display(df.iloc[listind[0]])
df.drop(listind, axis=0, inplace=True)
display(df.iloc[listind[0]])

review_sentences                                    
user_id             34216df2a5f3846b17e3f96bea6c2ad7
timestamp                                 2017-08-15
rating                                             3
has_spoiler                                    False
book_id                                      7818684
review_id           ee2b6a6ef25de1ddf311faedfee82357
Name: 1136, dtype: object

review_sentences    love harry potter story course would like thin...
user_id                              34216df2a5f3846b17e3f96bea6c2ad7
timestamp                                                  2016-05-22
rating                                                              5
has_spoiler                                                     False
book_id                                                      23734628
review_id                            d82af813eb4698f492eb2c3f9be644fb
Name: 1137, dtype: object

In [ ]:
print(len(listind))

342


In [ ]:
import torch
from torchtext.legacy import data
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.deterministic = True




In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 7.0 MB/s 
     |████████████████████████████████| 77 kB 8.3 MB/s 
     |████████████████████████████████| 895 kB 83.9 MB/s 
     |████████████████████████████████| 596 kB 74.8 MB/s 
     |████████████████████████████████| 6.5 MB 79.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

['hello', 'world', 'how', 'are', 'you', '?']
[7592, 2088, 2129, 2024, 2017, 1029]


In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)
fields={'has_spoiler':('label',LABEL),'review_sentences':('data',TEXT)}

In [ ]:
train,test=train_test_split(df,test_size=0.2)
test.head()
print(len(test))
del df

275539


In [ ]:
train.to_json('train.json',orient='records',lines=True)
test.to_json('test.json',orient='records',lines=True)
del train, test

In [ ]:
train_data,test_data=TabularDataset.splits(path='',
                                          train='train.json',
                                          test='test.json',
                                          format='json',
                                          fields=fields)

In [ ]:

print(len(train_data))

1102152


In [ ]:
import random
torch.cuda.empty_cache()
train_data, valid_data = train_data.split(random_state = random.seed(SEED))
MAX_VOCAB_SIZE = 25_000
print ("valid data sample", valid_data[0])

#TEXT.build_vocab(train_data,
#                 max_size = MAX_VOCAB_SIZE,
#                 vectors = "glove.6B.100d",
#                 unk_init = torch.Tensor.normal_)

"""TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE)"""
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)
train_iterator,valid_iterator,test_iterator=data.BucketIterator.splits((train_data,valid_data,test_data),sort=False,
                                                  batch_size=128,
                                                  device='cuda')

valid data sample <torchtext.legacy.data.example.Example object at 0x7f9efef0ca10>
defaultdict(None, {False: 0, True: 1})


In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(f'Number of valid 1 examples: {len(valid_data)}')
print(vars(valid_data.examples[0]))
print(vars(train_data.examples[0]))
print(test_data.examples[0].data)
#print(train_data.examples[0][1])
print(type(train_data.examples[0]))
#print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(LABEL.vocab)
#lstEmpty=[]

#for i in  range(len(valid_data)):
#if len(valid_data.examples[i].data)<=0:
#    lstEmpty.append(valid_data.examples[i].data)
 #   print(vars(valid_data.examples[i]))

#print(len(lstEmpty))


Number of training examples: 771506
Number of testing examples: 275539
Number of valid 1 examples: 330646
{'label': False, 'data': [2293, 2293, 3866, 7568, 3191, 2279, 2338, 6172, 4108, 2767, 2041, 3122, 4509, 11333, 17413, 12289, 9875, 3348, 2643, 7905, 27405, 5051, 2879, 2116, 2611, 5220, 2113, 3752, 2338, 2117, 2051, 2145, 25614, 2921, 3241, 4108, 8415, 2611, 8579, 4108, 16668, 6057, 2972, 2051, 3752, 2338, 3236, 4906, 17565, 27146, 1055, 13542, 2121, 1038, 20051, 4630, 14477, 22083, 9072, 7239, 7339, 2738, 8467, 7426, 2095, 2214, 2611, 3424, 6895, 17585, 2505, 2307, 5995, 9415, 4810, 4756, 25614, 2135, 10825, 2767, 2839, 2467, 2613, 2338, 2467, 2569, 4527, 9454, 2611, 6709, 4108, 9078, 6080, 2560, 2210, 2978, 2036, 2131, 2157, 9454, 2611, 2132, 4011, 6057, 2784, 15902, 2245, 4013, 22776, 21645, 8216, 2135, 17211, 2364, 17961, 3043, 2338, 2092, 2517, 14916, 8977, 2239, 2202, 5667, 3015, 25506, 2135, 6057, 2215, 2203, 3294, 2579, 2338, 4748, 5686, 2338, 4108, 2767, 14855, 2015, 2131,

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRU(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [ ]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,926,273 trainable parameters


In [ ]:
for name, param in model.named_parameters():
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 444,033 trainable parameters


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)


rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
out.weight
out.bias


In [ ]:
import torch.optim as optim

print(torch.__version__)
optimizer = optim.Adam(model.parameters())

criterion =nn.BCEWithLogitsLoss()
device = torch.device("cuda")
model = model.to(device)
criterion = criterion.to(device)


1.10.0+cu111


In [ ]:
def focal_loss(bce_loss, targets, gamma, alpha):
    """Binary focal loss, mean.

    Per https://discuss.pytorch.org/t/is-this-a-correct-implementation-for-focal-loss-in-pytorch/43327/5 with
    improvements for alpha.
    :param bce_loss: Binary Cross Entropy loss, a torch tensor.
    :param targets: a torch tensor containing the ground truth, 0s and 1s.
    :param gamma: focal loss power parameter, a float scalar.
    :param alpha: weight of the class indicated by 1, a float scalar.
    """
    p_t = torch.exp(-bce_loss)
    alpha_tensor = (1 - alpha) + targets * (2 * alpha - 1)  # alpha if target = 1 and 1 - alpha if target = 0
    f_loss = alpha_tensor * (1 - p_t) ** gamma * bce_loss
    return f_loss.mean()

In [ ]:
def binary_accuracy(rounded_preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer

    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(rounded_preds)
    return acc

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
!pip install torchmetrics

In [ ]:

from torchmetrics.classification import PrecisionRecallCurve

from torchmetrics import AUC
import torchmetrics
from torchmetrics import ConfusionMatrix
from torchmetrics import Accuracy


# initialize metric

def train(model, iterator, optimizer, criterion):
    aucMetric = AUC(compute_on_step=True,reorder=True)
    F1Metric=torchmetrics.F1Score(compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(compute_on_step=True).cuda()
    confmat = ConfusionMatrix(num_classes=2,compute_on_step=True).cuda()
    accTorch=Accuracy(compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0
    model.train()
    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.5, dtype = torch.float32).cuda()
    for batch in iterator:

        optimizer.zero_grad()

        #print(batch)
        #print(batch.data)

        predictions=model(batch.data).squeeze(1)
        predictionsigmoid=torch.sigmoid(predictions)
        rounded_preds=torch.round(predictionsigmoid)


        #print(rounded_preds)

        #print("Batch labels  " ,batch.label.int().cuda())
        bce_loss = criterion(predictions, batch.label)
        loss= focal_loss(bce_loss,batch.label,gamma,alpha)
        #print(loss)

        acc = binary_accuracy(rounded_preds, batch.label)
        confmat.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())
        accTorch.update(rounded_preds,batch.label.type(torch.IntTensor).cuda())
        aucMetric.update(rounded_preds, batch.label)
        F1Metric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())
        PrecMetric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())
        RecallMetric.update(rounded_preds,batch.label.type(torch.IntTensor).cuda())
        aucRCurveMetric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    allMetrics={'AUC':aucMetric.compute(),'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute(),'accTorch':accTorch.compute(),
                'Confusion':confmat.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics

In [ ]:
def evaluate(model, iterator, criterion):


    aucMetric = AUC(compute_on_step=True,reorder=True)
    F1Metric=torchmetrics.F1Score(compute_on_step=True).cuda()

    PrecMetric=torchmetrics.Precision(compute_on_step=True).cuda()
    RecallMetric=torchmetrics.Recall(compute_on_step=True).cuda()
    aucRCurveMetric=torchmetrics.AUROC(compute_on_step=True).cuda()
    epoch_loss = 0
    epoch_acc = 0
    epoch_auc=0

    gamma = torch.tensor(1.0, dtype = torch.float32).cuda()
    alpha=torch.tensor(0.3, dtype = torch.float32).cuda()
    model.eval()

    with torch.no_grad():

        for batch in  iterator:


            predictions = model(batch.data).squeeze(1)

            predictionsigmoid=torch.sigmoid(predictions)
            rounded_preds=torch.round(predictionsigmoid)


        #print(rounded_preds)

            #print("Batch labels  ", batch.label.int().cuda())
            bce_loss = criterion(predictions, batch.label)
            loss= focal_loss(bce_loss,batch.label,gamma,alpha)
        #print(loss)

            acc = binary_accuracy(rounded_preds, batch.label)
            aucMetric.update(rounded_preds, batch.label)
            F1Metric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())
            PrecMetric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())
            RecallMetric.update(rounded_preds,batch.label.type(torch.IntTensor).cuda())
            aucRCurveMetric.update(rounded_preds, batch.label.type(torch.IntTensor).cuda())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    allMetrics={'AUC':aucMetric.compute(),'Precision':PrecMetric.compute(),'Recall':RecallMetric.compute(),
                'F1':F1Metric.compute(),'AUCR':aucRCurveMetric.compute()}

    #allMetrics={'AUC':aucMetric.compute(),'F1':F1Metric.compute(),'Precision':PrecMetric.compute(),
    #            'Recall':RecallMetric.compute(),'AUCR':aucRCurveMetric.compute()}

    return epoch_loss / len(iterator), epoch_acc / len(iterator),allMetrics



In [ ]:

from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
writer = SummaryWriter('tensorboard/runs')
N_EPOCHS = 5
dfData=pd.DataFrame()
best_valid_loss = float('inf')
dfData=pd.DataFrame()
Epoch=[]
listDfData=[]
train_lossList,valid_lossList,test_lossList=[],[],[]
train_accList,valid_accList,test_accList=[],[],[]
train_aucList,valid_aucList,test_aucList=[],[],[]
train_auCuList,valid_auCuList,test_auCuList=[],[],[]
train_F1List,valid_F1List,test_F1List=[],[],[]
train_PreList,valid_PreList,test_PreList=[],[],[]
train_RecallList,valid_RecallList,test_RecallList=[],[],[]

for epoch in tqdm(range(N_EPOCHS)):

    start_time = time.time()

    train_loss, train_acc,train_allMetrics = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc,valid_allMetrics = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f} | All Metrics:{train_allMetrics}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f} | Val All Metrics:{train_allMetrics}')
    model.load_state_dict(torch.load('tut1-model.pt'))
    test_loss, test_acc,test_allMetrics= evaluate(model, test_iterator, criterion)
    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f} | Test All Metrics:{train_allMetrics}')
    Epoch.append(epoch)
    train_lossList.append(train_loss)
    valid_lossList.append(valid_loss)
    test_lossList.append(test_loss)
    train_accList.append(train_acc*100)
    valid_accList.append(valid_acc*100)
    test_accList.append(test_acc*100)

    train_aucList.append(train_allMetrics['AUC'].cpu().numpy())
    valid_aucList.append(valid_allMetrics['AUC'].cpu().numpy())
    test_aucList.append(test_allMetrics['AUC'].cpu().numpy())
    train_auCuList.append(train_allMetrics['AUCR'].cpu().numpy())
    valid_auCuList.append(valid_allMetrics['AUCR'].cpu().numpy())
    test_auCuList.append(test_allMetrics['AUCR'].cpu().numpy())
    train_F1List.append(train_allMetrics['F1'].cpu().numpy())
    valid_F1List.append(valid_allMetrics['F1'].cpu().numpy())
    test_F1List.append(test_allMetrics['F1'].cpu().numpy())
    train_PreList.append(train_allMetrics['Precision'].cpu().numpy())
    valid_PreList.append(valid_allMetrics['Precision'].cpu().numpy())
    test_PreList.append(test_allMetrics['Precision'].cpu().numpy())
    train_RecallList.append(train_allMetrics['Recall'].cpu().numpy())
    valid_RecallList.append(valid_allMetrics['Recall'].cpu().numpy())
    test_RecallList.append(test_allMetrics['Recall'].cpu().numpy())
dfData.insert(0, 'Epoch', Epoch)
dfData.insert(1, 'train_loss_', train_lossList)
dfData.insert(2, 'valid_loss_', valid_lossList)
dfData.insert(3, 'test_loss_', test_lossList)
dfData.insert(4, 'train_acc_', train_accList)
dfData.insert(5, 'valid_acc_', valid_accList)
dfData.insert(6, 'test_acc_', test_accList)
dfData.insert(7, 'train_auc_', train_aucList)
dfData.insert(8, 'valid_auc_', valid_aucList)
dfData.insert(9, 'test_auc_', test_aucList)
dfData.insert(10, 'train_AUCR_', train_auCuList)
dfData.insert(11, 'valid_AUCR_', valid_auCuList)
dfData.insert(12, 'test_AUCR_', test_F1List)
dfData.insert(13, 'train_F1_', train_F1List)
dfData.insert(14, 'valid_F1_', valid_F1List)
dfData.insert(15, 'test_F1_', test_auCuList)
dfData.insert(16, 'train_Precision_', train_PreList)
dfData.insert(17, 'valid_Precision_', valid_PreList)
dfData.insert(18, 'test_Precision_', test_PreList)
dfData.insert(19, 'train_Recall_', train_RecallList)
dfData.insert(20, 'valid_Recall_', valid_RecallList)
dfData.insert(21, 'test_Recall_', test_RecallList)



  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [ ]:
pip install xlsxwriter

In [ ]:
writer.close()
dfData.head()

In [ ]:
from datetime import datetime
import xlsxwriter
from xlrd import open_workbook
import openpyxl
import xlwt
import os.path
filename=datetime.now().strftime("%d-%b-%Y")+".xlsx"
sheetName=datetime.now().strftime("%H-%M-%S")
workbook = xlsxwriter.Workbook(pathResutlts + filename)
if os.path.isfile(pathResutlts+filename):
    print('old file')
    ExcelWorkbook = openpyxl.load_workbook(pathResutlts+filename)

    # Generating the writer engine
    writer = pd.ExcelWriter(pathResutlts+filename, engine = 'openpyxl')

    # Assigning the workbook to the writer engine
    writer.book = ExcelWorkbook

    dfData.to_excel(writer, sheet_name = sheetName)

    writer.save()
else:

    # saving the excel
    dfData.to_excel(pathResutlts+filename,sheet_name=sheetName)
    print('DataFrame is written to Excel File successfully.')

